<a href="https://colab.research.google.com/github/ramesh8/ner-train-google-colab/blob/main/learningd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy

In [ ]:
!pip install --upgrade spacy


In [ ]:
!pip install -U 'spacy[cuda-autodetect]'

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.5.0/en_core_web_lg-3.5.0.tar.gz

In [ ]:
import spacy
gpu = spacy.prefer_gpu()
print(gpu)
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is Ramesh")

for ent in doc.ents:
  print(ent.text, ent.label_)

In [ ]:
!nvcc --version

In [ ]:
import locale
print(locale.getpreferredencoding())

In [8]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from google.colab import drive

drive.mount("/drive")

In [ ]:
import json
import zipfile

json_data = []
with zipfile.ZipFile("/drive/My Drive/ML/NLP/NER/entities.zip", "r") as z:
  for fname in z.namelist():
    print(fname)
    with z.open(fname) as f:
      data = f.read()
      json_data = json.loads(data.decode("utf-8"))
      print(f"{len(json_data)} entities found" )

for i, group in enumerate(grouper(json_data, split_size)):
    with open(f'splits\entities_split_{i}.json', 'w') as opfile:
        groupft = [gi for gi in list(group) if gi]        
        json.dump(groupft, opfile)

In [ ]:
import json
from itertools import zip_longest

split_size = 500

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

In [ ]:
import json
import zipfile

json_data = []
with zipfile.ZipFile("/drive/My Drive/ML/NLP/NER/entities.zip", "r") as z:
  for fname in z.namelist():
    print(fname)
    with z.open(fname) as f:
      data = f.read()
      json_data = json.loads(data.decode("utf-8"))
      print(f"{len(json_data)} entities found" )

for i, group in enumerate(grouper(json_data, split_size)):
    with open(f'/drive/My Drive/ML/NLP/NER/splits/entities_split_{i}.json', 'w') as opfile:
        groupft = [gi for gi in list(group) if gi]        
        json.dump(groupft, opfile)

In [ ]:
!python -m spacy convert "/content/drive/MyDrive/ML/NLP/NER/splits/" "/content/drive/MyDrive/ML/NLP/NER/spacy/" 

In [ ]:
!pip install tensorrt

In [ ]:
import os
splitpath = "/drive/My Drive/ML/NLP/NER/splits/"
for file in os.listdir(splitpath):
  jsonfile = os.path.join(splitpath, file)
  targetfile = os.path.splitext(jsonfile)[0]+".spacy"
  print(jsonfile, targetfile)

In [4]:
import spacy
from spacy.tokens import DocBin
import json
import time
import string

class trainer:
    def __init__(self,tdurl='data\entities.json',spacy_file='train.spacy'):
        self.file_url = tdurl
        self.training_data_json = []
        self.training_data = []
        self.spacy_file_name = spacy_file
        self.success_counter = 0
        self.fail_counter = 0

    def load_training_data(self):
        try:
            with open(self.file_url,mode='r',encoding='utf-8') as f:
                self.training_data_json = json.load(f)
                return True
        except Exception as ex:
            print("Failed to Load Trianing Data")
            return False

    def prepare_training_data(self):
        self.failed_list = []
        if len(self.training_data_json)==0:
            print("No Training Data")
            return False
        self.training_data = []
        noentscnt = 0
        dupscnt = 0
        for tdj in self.training_data_json:
            _, pid, text,entities = tdj.values()
            if len(entities)==0:
                noentscnt += 1
                continue
            for i, ent in enumerate(entities):
                if ent[0] == ent[1]:
                    del entities[i]
            if len(entities)==0:
                noentscnt += 1
                continue

            ent = [tuple(en) for en in entities]
            old_len = len(ent)
            ent = list(set(ent))
            new_len = len(ent)
            self.training_data.append((text, {"entities":ent}))
            if old_len != new_len:
                dupscnt += (old_len-new_len)    

    def convert_to_spacy_old(self):
        self.fail_to_convert_list = []        
        invalidChars = set(string.punctuation.replace("_", " "))
        nlp = spacy.load("en_core_web_lg")
        # nlp = spacy.blank("en")
        db = DocBin()    
        skipped = 0
        total = 0
        for text, annotations in self.training_data:
            doc = nlp.make_doc(text)
            ents = []
            
            for start, end, label in annotations['entities']:
                substr = text[start:end]
                span = doc.char_span(start, end, label=label, alignment_mode="expand")
                total += 1
                if span is None:
                    skipped += 1
                elif span.text.startswith(' ') or span.text.endswith(' '):
                    skipped += 1
                else:
                    ents.append(span)            
            _ents = list(set(ents))
            try:    
                doc.ents = _ents
                db.add(doc)   
                self.success_counter += 1             
            except Exception as ex:
                self.fail_counter += 1
            
        db.to_disk(self.spacy_file_name)
        print(self.success_counter, self.fail_counter)


In [ ]:
import os
splitpath = "/content/drive/MyDrive/ML/NLP/NER/splits/"
for file in os.listdir(splitpath):
  jsonfile = os.path.join(splitpath, file)
  targetfile = os.path.splitext(jsonfile)[0]+".spacy"
  targetfile = targetfile.replace("splits","spacy")
  tr = trainer(jsonfile,targetfile) 
  tr.load_training_data()
  tr.prepare_training_data()
  tr.convert_to_spacy_old()

In [ ]:
!pwd

In [ ]:
cd drive/MyDrive/ML/NLP/NER/

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train --paths.dev ./dev --gpu-id=0

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train --paths.dev ./dev